Here we are performing preprocessing with the MUSE data. 
We need to take care of the "nan" (Dead pixel) and rotate the image cube.

In [ ]:
# Import library

# Packages required
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.io import fits
import os
from deconvbench import Deconvbench
from mpl_toolkits.axes_grid1 import make_axes_locatable
from amiral import instructment, utils, parameter, gradient, minimisation, array, plotting
from astropy.visualization import make_lupton_rgb
import scipy
import astropy.io as astropy
import maoppy


# from plotting import plot_PSF_PSD as amiral_plt
from scipy.optimize import minimize 

%matplotlib inline

import tools

rcParams["figure.figsize"] = 20,33

In [ ]:
# Function for getting snr
def snr_map (array, n_sky, n_ron): 
    
    # S/N = S/N_tot = S /  sqrt (S+Sky+Dark +N_ron^2)
    # As the RON is small, we can ignore that 
    # Sky: background -> get the mean value of the background from the corner
    
    dimension = np.shape(array)[0]
    snr_map =  np.zeros((dimension,dimension))
    snr_map[:,:] = array[:,:] / np.sqrt(array[:,:] + n_sky + n_ron**2)
    
    return snr_map

def get_snr (array):
    
    mean = np.mean(array)
    sig2 = np.std(array)
    
    snr = mean / sig2
    
    return snr

def resize_array (array, size, cent = None):
    """
    
    Resize the array to a given size. Cent is optional.
    
    If cent is an input, it contains the centre of the image
    
    """
    if array.ndim == 3: 
        if cent == None:
            cent = [np.shape(array[0])[0]//2, np.shape(array[0])[1]//2]
            
        _len = np.shape(array)[0]
        resize_cube = np.zeros((_len,size, size))
        
        for i in range (_len):
            resize_cube[i] = array[i][cent[0]- size//2:cent[0] +size//2,\
                                      cent[0] - size//2:cent[0] +size//2]
        return resize_cube
    
    else:
        if cent == None:
            cent = [np.shape(array)[0]//2, np.shape(array)[1]//2]
       
        resize_array = array[cent[1] - size//2:cent[1] +size//2, \
                             cent[0] - size//2:cent[0] +size//2]

        return resize_array

def drop_frame (cube, value):
    """
    Get the indice for array which contains too many "nan" values
    """
    dropped_list = []
    
    for i in range (len(cube)):
    
        nan_ind = np.argwhere(np.isnan(cube[i]))
        if len(nan_ind) > value:
            dropped_list.append(i)
        
    return dropped_list

def moffat_fwhm (alpha, beta): 
    
    return 2.*alpha*np.sqrt(np.power(2, 1/beta) - 1)


from astropy.modeling import models, fitting
from astropy.stats import gaussian_sigma_to_fwhm,gaussian_fwhm_to_sigma
import photutils



def gauss2D_fitting (array, fwhm = (4,4), theta=0, full_output= False):
    
    # Following the definitions in VIP
    
    _fwhmx,_fwhmy = fwhm

    x_cent, y_cent = photutils.centroid_com(array)
    
    # gives you the range 
    init_amp = np.ptp(PSF[0]) 

    gauss = models.Gaussian2D(amplitude=init_amp, theta=0,
                              x_mean=x_cent, y_mean=y_cent,
                              x_stddev=_fwhmx * gaussian_fwhm_to_sigma,
                              y_stddev=_fwhmy * gaussian_fwhm_to_sigma)


    fitter = fitting.LevMarLSQFitter()
    y, x = np.indices(array.shape)
    fit = fitter(gauss, x, y, PSF[0])

    mean_y = fit.y_mean.value
    mean_x = fit.x_mean.value
    fwhm_y = fit.y_stddev.value*gaussian_sigma_to_fwhm
    fwhm_x = fit.x_stddev.value*gaussian_sigma_to_fwhm
    amplitude = fit.amplitude.value
    theta = np.rad2deg(fit.theta.value)
    
    
    if full_output == True: 
        fit_frame = pd.DataFrame({'centriod_x': mean_x, 'centriod_y': mean_y, 
                             'fwhm_y': fwhm_y, 'fwhm_x': fwhm_x,
                             'amplitude': amplitude, 'theta': theta})
        return fit_frame 
    else: 
        return fwhm_x, fwhm_y


In [ ]:
# PATH
wdir = "/Users/alau/Data/MUSE_DATA/HD_146233/"
data_cube = ["HD_146233_cube_1_binned_10","HD_146233_cube_2_binned_10"]

In [ ]:
# Variable
muse_pix_scale = 0.025 # arcsecond per pixel

Have a look at the actual data before performing any data analysis

In [ ]:
name = data_cube[1]

_cube = fits.open(wdir+name+".fits")
_cube.info()

cube = _cube[1].data



For now, we can see that we still have nan values in our data. We need to filter them before performing fitting.

In [ ]:
cube_masked = np.ma.masked_invalid(cube)

In [ ]:
# Parameters for maoppy 
cent_x = 105 # xmin
cent_y = 111 # ymin
Npix = np.shape(cube[0])[0]
Nslice = np.shape(cube)[0]

wvl_min = _cube[1].header['CRVAL3']*1e-10/1e-9
wvl_slice = _cube[1].header['CD3_3']*1e-10/1e-9

print("Starting wavelength: %.2f [nm]" %(wvl_min))
print("Wavelength Slice: %.2f [nm]" %(wvl_slice))
print("Number of slice: %d" %(Nslice))


PSF = cube_masked

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from maoppy.psfmodel import Psfao, psffit
from maoppy.instrument import muse_nfm

In [ ]:
wvl = wvl_min*1e-9
samp = muse_nfm.samp(wvl)

Pmodel = Psfao((Npix,Npix),system=muse_nfm,samp=samp)


#%% Generate a MUSE-NFM PSF
r0 = 0.15 # Fried parameter [m]
b = 1e-7 # Phase PSD background [rad² m²]
amp = 1.4 # Phase PSD Moffat amplitude [rad²]
alpha = 0.1 # Phase PSD Moffat alpha [1/m]
ratio = 1.0
theta = 0.0
beta = 1.6 # Phase PSD Moffat beta power law


In [ ]:
param = [r0,b,amp,alpha,ratio,theta,beta]

psf = Pmodel(param,dx=0,dy=0)

# noise = ron*np.random.randn(npix,npix)
# image = flux*psf + bckgd + noise

#%% Fit the PSF with Psfao

weights = 1.0/(PSF[0]*(PSF[0]>0)+muse_nfm.ron**2) #np.ones_like(PSF[i,...])
bad = np.where(PSF[0].mask == True) # all columns and rows in [i]
weights[bad[0],bad[1]] = 0.

guess = [0.145,2e-7,1.2,0.08,ratio,theta,1.5]
fixed = [False,True,False,False,True,True,False]

out = psffit(PSF[0],Psfao,guess,weights,system=muse_nfm,samp=samp,fixed=fixed)



In [ ]:
flux_fit, bck_fit = out.flux_bck
fitao = flux_fit*out.psf + bck_fit

#%% Plots
print(31*'-')
print("%9s %10s %10s"%('','MUSE-NFM','Psfao'))
print(31*'-')
print("%9s %10.4g %10.4g"%('Flux [e-]',np.sum(PSF[0]),flux_fit))
# print("%9s %10.4g %10.4g"%('Bck [e-]',bckgd,bck_fit))
print("%9s %10.2f %10.2f"%('r0 [cm]',r0*100,out.x[0]*100))
print("%9s %10.2f %10.2f"%('A [rad²]',amp,out.x[2]))
print(31*'-')

In [ ]:
vmin,vmax = np.arcsinh([PSF[0].min(),PSF[0].max()])

xlimt = 200//2 * muse_pix_scale
ylimt = 200//2 * muse_pix_scale

extent = [-xlimt,xlimt,-ylimt,ylimt]

fig, ax = plt.subplots(1,3,constrained_layout=True, figsize=(20, 10))

pos = ax[0].imshow(np.log10(PSF[0]),interpolation='none',vmin=vmin,vmax=vmax,extent=extent)
fig.colorbar(pos, ax=ax[0], pad = 0.15, shrink = 0.5)
ax[0].set_title('HD_146233',fontsize = 16)
ax[0].set_xlabel(r'$\mathrm{arcsecond}$', fontsize = 14)

pos = ax[1].imshow(np.log10(fitao),vmin=vmin,vmax=vmax,extent=extent)
fig.colorbar(pos, ax=ax[1], pad = 0.15, shrink = 0.5)

ax[1].set_title('Psfao fit',fontsize = 16)
ax[1].set_xlabel(r'$\mathrm{arcsecond}$', fontsize = 14)

pos= ax[2].imshow(np.log10(PSF[0]-fitao),extent=extent)
fig.colorbar(pos, ax=ax[2], pad = 0.15, shrink = 0.5)

ax[2].set_title('residuals',fontsize = 16)
ax[2].set_xlabel(r'$\mathrm{arcsecond}$', fontsize = 14)

In [ ]:
plt.imshow(PSF[0]-fitao)

In [ ]:
# fig, ax = plt.subplots(1,3,constrained_layout=True, figsize=(20, 10))

# pos = ax[0].imshow(np.arcsinh(resize_PSF),interpolation='none',vmin=vmin,vmax=vmax,extent=extent)
# fig.colorbar(pos, ax=ax[0], pad = 0.15, shrink = 0.5)
# ax[0].set_title('HD_146233',fontsize = 16)
# ax[0].set_xlabel(r'$\mathrm{arcsecond}$', fontsize = 14)

# pos = ax[1].imshow(np.arcsinh(resize_fitao),vmin=vmin,vmax=vmax,extent=extent)
# fig.colorbar(pos, ax=ax[1], pad = 0.15, shrink = 0.5)

# ax[1].set_title('Psfao fit',fontsize = 16)
# ax[1].set_xlabel(r'$\mathrm{arcsecond}$', fontsize = 14)

# pos= ax[2].imshow(np.arcsinh(resize_PSF-resize_fitao),extent=extent)
# fig.colorbar(pos, ax=ax[2], pad = 0.15, shrink = 0.5)

# ax[2].set_title('residuals',fontsize = 16)
# ax[2].set_xlabel(r'$\mathrm{arcsecond}$', fontsize = 14)

In [ ]:
from amiral.plotting import plotting_PSF_PSD
# Plot the PSF slice
rcParams['figure.figsize'] = 11,9

xlimt = 128//2 * muse_pix_scale
ylimt = 128//2 * muse_pix_scale



resize_PSF = resize_array(PSF[0]/np.sum(PSF[0]),128,cent=(105,111))
resize_fitao = resize_array(fitao/np.sum(fitao),128,cent=(105,111))

mean_PSF = utils.mean_cir_array(resize_PSF)
mean_fit = utils.mean_cir_array(resize_fitao)

x_arcsecond = muse_pix_scale*np.linspace(0,len(mean_PSF),len(mean_PSF))

fig, ax = plt.subplots()
ax.set_title(r"PSF (mean)"
             "\nwvl=%.2f nm" %(wvl*1e9),fontsize = 18)
ax.set_yscale('log')
ax.set_ylabel(r'$\mathrm{Mean Intensity [e^-]}$', fontsize = 18)
ax.set_xlabel(r'$\mathrm{arcsecond}$', fontsize = 18)
ax.plot(x_arcsecond,mean_PSF, label = "True")
ax.plot(x_arcsecond,mean_fit, label = "Estimated PSF")
ax.tick_params(axis='both', which='major', labelsize=18)
ax.legend()



In [ ]:
x_arcsecond = muse_pix_scale*np.linspace(-128//2,128//2,128)

rcParams['figure.figsize'] = 11,9
fig, ax = plt.subplots(1)
ax.set_yscale('log')
ax.set_ylabel(r'$\mathrm{Intensity [e^-]}$', fontsize = 18)
ax.set_xlabel(r'$\mathrm{arcsecond}$', fontsize = 18)
ax.plot(x_arcsecond,np.abs(resize_PSF[64,...]),label = "Image")
ax.plot(x_arcsecond,np.abs(resize_fitao[64,...]),label = "Fit")

# ax.set_title(r"PSF Estimation (with noise)$\mathrm{ \{r_0, sig^2, mu, \rho_0 \}}$"
#              "\nSR Error: %.2f %%\nFlux[e-]: %.2E" %(0,0),fontsize = 18)
ax.set_title(r"PSF cut"
             "\nwvl=%.2f nm" %(wvl*1e9),fontsize = 18)
ax.tick_params(axis='both', which='major', labelsize=18)
ax.legend()
print(wvl)

Find the fwhm of the PSF from the data --> do the same for the estimated PSF

In [ ]:
out.x[0]

In [ ]:
out_alpha = out.x[3]
out_beta = out.x[-1]

rad2arcsecond = 206265

fwhm_fitted = moffat_fwhm(out_alpha,out_beta)
print("wvl: %.2e " %(wvl))
print("Sampling: %.2e [pix/arcseond]" %(rad2arcsecond*wvl/(muse_nfm.D*muse_pix_scale)))
print("FWHM of the fitted PSF (Moffat): %.2e [mas]" %(1000*fwhm_fitted*muse_pix_scale))
print("Estimated Seeing: %.2f [mas]" %(1000*rad2arcsecond*wvl/(out.x[0])))

In [ ]:
# # #%% PLOTS
# wvl = np.arange(Nslice)*wvl_slice + wvl_min
# laser_min = 575 -5 # [rjlf] approximative wavelength of the dichroic on MUSE due to LGS
# laser_max = 595 +7

# index = np.where(R0 > 0)[0] # [rjlf] do not account for r0 that where not properly fitted

# # [rjlf] Fit all the r0 found with the theoretical curve: r0 ~ wvl^(6/5)
# def cost(r):
#     return R0[index] - r*(wvl[index]/500.*1e9)**(6./5.)

# roptim = least_squares(cost,0.13).x[0]

# plt.figure(1)
# plt.clf()
# ax = plt.subplot(111)
# plt.plot(wvl[index]*1e9,(roptim*(wvl[index]/500.*1e9)**(6./5.))*100.,color="C1",linewidth=4,label="Theory",zorder=0)
# plt.scatter(wvl[index]*1e9,R0[index]*100.,label="Fit",zorder=1,s=25)
# #plt.xlabel("Lambda [nm]",fontsize=fs)
# #plt.ylabel("r0 [cm]",fontsize=fs)
# plt.ylim(5,25)
# plt.xlim(400,950)
# plt.grid()
# #plt.fill([laser_min,laser_max,laser_max,laser_min],[plt.ylim()[0],plt.ylim()[0],plt.ylim()[1],plt.ylim()[1]],color="gray",alpha=.5)
# #plt.title("Fried parameter")
# plt.legend()
# plt.xticks()
# plt.yticks()
# #plt.fill([laser_min,laser_max,laser_max,laser_min],[plt.ylim()[0],plt.ylim()[0],plt.ylim()[1],plt.ylim()[1]],color="gray",alpha=.5)

# # print("r0[500nm] = %.2f cm"%(roptim*100))

# # plt.legend(fontsize=20)
# # plt.xticks(fontsize=20)
# # plt.yticks(fontsize=20)
# # plt.xlabel('$\lambda$ [nm]',fontsize=20)
# # plt.ylabel('$r_0$ [cm]',fontsize=20)


# # plt.figure(2)
# # plt.clf()

# # index = np.max(np.arange(Nslice)*((wvl*1e9)<laser_min))+1
# # plt.plot(wvl[1:index]*1e9,AMP[1:index]*1e-8,color='C0',linewidth=2,label='Flux [1e8]')
# # plt.plot(wvl[1:index]*1e9,BCK[1:index],color='C1',linewidth=2,label='bck')

# # index = np.max(np.arange(Nslice)*((wvl*1e9)<laser_max))+1
# # plt.plot(wvl[index:]*1e9,AMP[index:]*1e-8,color='C0',linewidth=2)
# # plt.plot(wvl[index:]*1e9,BCK[index:],color='C1',linewidth=2)

# # plt.grid()
# # plt.xticks(plt.xticks()[0])
# # plt.xlabel("Lambda [nm]")
# # plt.ylabel("Flux & Bck [photon]")
# # plt.legend()
# # plt.xlim(wvl[1]*1e9,wvl[-1]*1e9)



# # plt.figure(4)
# # plt.clf()
# # cmap = ['Blues','Greens','Reds']
# # nb = 3
# # it = 1

# # # [rjlf] These 3 lines can be removed, they don't seem to be used
# # x = np.arange(len(circavg(PSF[0,...])))*1.0
# # x[0] = 1e-1
# # xPSD = None

# # for i in range(1,Nslice-nb,int(Nslice/nb)):
    
# #     PSFplot = np.arcsinh(PSF[i,...])
# #     FITplot = np.arcsinh(FIT[i,...])
# #     diff = np.arcsinh(PSF[i,...]-FIT[i,...])
    
# #     cmin = min([PSFplot.min(),FITplot.min()])
# #     cmax = max([PSFplot.max(),FITplot.max()])
# #     #cmin = -cmax
    
# #     plt.subplot(3,nb,it)
# #     #plt.title("wvl = %unm"%(wvl[i]*1e9))
# #     plt.pcolormesh(PSFplot,cmap=cmap[it-1])
# #     plt.axis('image')
# #     plt.axis('off')
# #     plt.clim(cmin,cmax)
    
# #     plt.subplot(3,nb,nb+it)
# #     plt.pcolormesh(FITplot,cmap=cmap[it-1])
# #     plt.axis('image')
# #     plt.axis('off')
# #     plt.clim(cmin,cmax)
    
# #     plt.subplot(3,nb,2*nb+it)
# #     plt.pcolormesh(diff,cmap=cmap[it-1])
# #     plt.axis('image')
# #     plt.axis('off')
# #     plt.clim(cmin,cmax)
    
# #     it+=1

# # plt.tight_layout()
# # #plt.savefig(folderOUT+'psf_muse.png', transparent=True, overwrite=True)

# # # [rjlf] This plot was useful since my PSF was centered
# # # for this specific data, the center moves with the wavelength!!!
# # plt.figure(5)
# # plt.clf()
# # cmap = 'hot'
# # nb = 3
# # it = 1
# # for i in range(1,Nslice-nb,int(Nslice/nb)):

# #     center = (Npix//2,Npix//2)
    
# #     plt.subplot(1,3,it)
# #     x,y = circavgplt(PSF[i,...],center=center)
# #     plt.semilogy(x,y,label="PSF")
# #     xm,ym = circavgplt(FIT[i,...],center=center)
# #     plt.semilogy(xm,ym,label="MODEL")
# #     xp,yp = circavgplt(np.abs(PSF[i,...]-FIT[i,...]),center=center)
# #     plt.semilogy(xp,(yp),label="|PSF-MODEL|")
    
# #     plt.xlim(-50,50)
# #     plt.ylim(1e2,6e8)
# #     plt.title("wvl = %unm"%(wvl[i]*1e9))
# #     plt.legend()
# #     it+=1

# # #%% ERRORS
# # def norm(x,p=1.0,axis=(1,2)):
# #     if p<=0: raise ValueError('`p` should be strictly positive')
# #     return (np.sum(np.abs(x)**p,axis=axis))**(1.0/p)

# # good = PSF > (-50)
# # ERR = norm((FIT-PSF)*good)/np.sum(PSF*good,axis=(1,2))
# # ERRMOFF = norm((FITMOFF-PSF)*good)/np.sum(PSF*good,axis=(1,2))

# # noise = np.random.randn(*PSF.shape) * muse_nfm.ron
# # ERRN = norm(noise)/np.sum(PSF*good,axis=(1,2))

# # # aberrant slices
# # ERR[0] = np.nan
# # ERR[25] = np.nan
# # ERRMOFF[0] = np.nan
# # ERRMOFF[25] = np.nan
# # ERRN[0] = np.nan
# # ERRN[25] = np.nan

# # v = np.where(1-np.isnan(ERR))
# # p = np.polyval(np.polyfit(wvl[v],ERR[v],1),wvl)
# # pm = np.polyval(np.polyfit(wvl[v],ERRMOFF[v],1),wvl)

# # plt.figure(6)
# # plt.clf()

# # plt.plot(wvl*1e9,ERR*100,label='PSFAO',lw=3)
# # plt.plot(wvl*1e9,ERRMOFF*100,label='Moffat',lw=3)
# # #plt.plot(wvl*1e9,ERRN*100,label='Noise',lw=1)

# # plt.plot(wvl*1e9,p*100,lw=3,ls=':')
# # plt.plot(wvl*1e9,pm*100,lw=3,ls=':')

# # plt.legend(loc='lower right')
# # plt.xlabel('$\lambda$ [nm]')
# # plt.ylabel('Error [%]')
# # plt.ylim(0,30)
# # plt.xlim(wvl.min()*1e9,wvl.max()*1e9)
# # plt.grid()



In [ ]:
# Passing parametpsd_arrayers from the telesope to the aosystem
aosys_cls = instructment.aoSystem( 
        diameter = muse_nfm.D, occ_ratio =muse_nfm.occ, 
        no_acutuator= 39, wavelength = wvl, 
        resolution_rad = 1.1977272727272726e-07, 
        dimension=Npix)  

pupil = aosys_cls.get_pupil_plane()
otf_tel = aosys_cls.pupil_to_otf_tel(pupil)


cutoff_freq = aosys_cls.ao_cutoff_freq
print(cutoff_freq)


psf_tel = np.real(np.fft.ifft2(np.fft.ifftshift(otf_tel)))
print("Sum of the psf_tel :%.2f" %(np.sum(psf_tel)))

print(psf_tel.shape)

In [ ]:
fXY = np.mgrid[0:128, 0:128].astype(float)


df = aosys_cls.get_pix2freq(aosys_cls.samp_factor)
print(df)



fX = fXY[0] - 128//2
fY = fXY[1] - 128//2

print(fX.shape)

fX *= df
fY *= df

fX[:,64][64:]

In [ ]:
_ind = np.where(PSF[0].mask == True)

PSF[0][_ind] = 0

In [ ]:
# Plot the PSF slice
rcParams['figure.figsize'] = 11,9

xlimt = 128//2 * muse_pix_scale
ylimt = 128//2 * muse_pix_scale

otf_image = np.fft.fftshift(np.fft.fft2(resize_PSF))
otf_fit = np.fft.fftshift(np.fft.fft2(resize_fitao))

mean_otf_image = np.abs(utils.mean_cir_array(np.real(otf_image)))
mean_otf_fit = np.abs(utils.mean_cir_array(np.real(otf_fit)))

x_arcsecond = muse_pix_scale*np.linspace(0,len(mean_PSF),len(mean_PSF))

fig, ax = plt.subplots()
ax.set_yscale('log')
ax.set_title(r"OTF(Mean)"
             "\nwvl=%.2f nm" %(wvl*1e9),fontsize = 18)

ax.set_ylabel(r'$\mathrm{MeanIntensity [e^-]}$', fontsize = 18)
ax.set_xlabel(r'$\mathrm{Pixel}$', fontsize = 18)
ax.plot(mean_otf_image, label = "Image")
ax.plot(mean_otf_fit, label = "Estimated")
ax.tick_params(axis='both', which='major', labelsize=18)
ax.legend()

In [ ]:
# Plot the PSF slice
rcParams['figure.figsize'] = 11,9

xlimt = 128//2 * muse_pix_scale
ylimt = 128//2 * muse_pix_scale

otf_image = np.fft.fftshift(np.fft.fft2(resize_PSF))
otf_fit = np.fft.fftshift(np.fft.fft2(resize_fitao))

mean_otf_image = utils.mean_cir_array(np.abs(otf_image))
mean_otf_fit = utils.mean_cir_array(np.abs(otf_fit))

x_arcsecond = muse_pix_scale*np.linspace(0,len(mean_PSF),len(mean_PSF))

fig, ax = plt.subplots()
ax.set_yscale('log')
ax.set_title(r"OTF Cut"
             "\nwvl=%.2f nm" %(wvl*1e9),fontsize = 18)
ax.set_ylabel(r'$\mathrm{Intensity [e^-]}$', fontsize = 18)
ax.set_xlabel(r'$\mathrm{Frequency [m^{-1}]}$', fontsize = 18)
ax.plot(fX[:,64][64:],np.abs(otf_image)[64,64:], label = "Image")
ax.plot(fX[:,64][64:],np.abs(otf_fit)[64,64:], label = "Estimated")
ax.tick_params(axis='both', which='major', labelsize=18)
ax.legend()

In [ ]:
_ind = np.where(PSF[0].mask == True)

PSF[0][_ind] = 0

otf_image = np.fft.fftshift(np.fft.fft2(PSF[0]))

fig, ax = plt.subplots(1,2)
ax[0].set_title(r'HD_146233''\nwvl = %.2fnm' %(wvl*1e9),fontsize = 16)
ax[0].imshow(np.real(np.log10(otf_image)))
ax[1].set_title(r'Psfao fit''\nwvl = %.2fnm' %(wvl*1e9),fontsize = 16)
ax[1].imshow(np.real(np.log10(otf_fit)))
# ax[2].imshow(np.real(np.log10(otf_image-otf_fit)))

In [ ]:
# Plot the PSF slice
rcParams['figure.figsize'] = 11,9

xlimt = 200//2 * muse_pix_scale
ylimt = 200//2 * muse_pix_scale

otf_image = np.fft.fftshift(np.fft.fft2(PSF[0]/np.sum(PSF[0])))
otf_fit = np.fft.fftshift(np.fft.fft2(fitao/np.sum(fitao)))

x_arcsecond = muse_pix_scale*np.linspace(0,len(mean_PSF),len(mean_PSF))

fig, ax = plt.subplots()
ax.set_title(r"OTF Slice"
             "\nwvl=%.2f nm" %(wvl*1e9),fontsize = 18)
ax.set_yscale('log')
ax.set_ylabel(r'$\mathrm{Mean Intensity [e^-]}$', fontsize = 18)
ax.set_xlabel(r'$\mathrm{Pixel}$', fontsize = 18)
ax.plot(np.abs(otf_image[100,...]), label = "Image")
ax.plot(np.abs(otf_fit[100,...]), label = "Estimated PSF")
ax.tick_params(axis='both', which='major', labelsize=18)
ax.legend()



In [ ]:
from maoppy.utils import binning


psf_tel_binned = binning (psf_tel, aosys_cls.samp_factor[0])

print("Sum of the psf_tel :%.2f" %(np.sum(psf_tel_binned)))



In [ ]:
100*np.max(PSF[0]/np.sum(PSF[0]))/np.max(psf_tel_binned)

In [ ]:
100*np.max(fitao/np.sum(fitao))/np.max(psf_tel_binned)

In [ ]:
np.max(PSF[0])

In [ ]:
np.max(fitao)

In [ ]:
fwhm_x,fwhm_y = gauss2D_fitting(PSF[0])

print(fwhm_x,fwhm_y)
print(1000*muse_pix_scale*((fwhm_x+fwhm_y)/2))

In [ ]:
fwhm_x,fwhm_y = gauss2D_fitting(fitao)

print(aosys_cls.sampling)
print(fwhm_x,fwhm_y)
print(1000*muse_pix_scale*((fwhm_x+fwhm_y)/2))

In [ ]:
np.max(PSF[0]/np.sum(PSF[0]))

In [ ]:
print(np.std(PSF[0]))

In [ ]:
import pandas as pd

path = "/Users/alau/Repo/amiral_data_analysis/"
name = ["HD_146233_cube_1.csv","test.csv" ]

df1 = pd.read_csv(path+name[0])
df2 = pd.read_csv(path+name[1])


df = [df1,df2]

conbined_df = pd.concat(df).reset_index(drop=True)

In [ ]:
fitted_r0 = conbined_df['r0']
wvl_list = conbined_df['wvl']


index = np.where(fitted_r0 > 0)[0] 

print(df1['r0']-df2['r0'])

In [ ]:
from scipy.optimize import least_squares

def cost(r):
    return fitted_r0[index] - r*(wvl_list[index]/500.*1e9)**(6./5.)

roptim = least_squares(cost,0.13).x[0]




# Plot the PSF slice
rcParams['figure.figsize'] = 11,9

fig, ax = plt.subplots()
ax.set_ylabel(r'$\mathrm{r0 [cm]}$', fontsize = 18)
ax.set_xlabel(r'$\mathrm{wavelength [nm]}$', fontsize = 18)
ax.plot(wvl_list[index]*1e9,(roptim*(wvl_list[index]/500.*1e9)**(6./5.))*100.,
        color="C1",linewidth=4,label="Theory",zorder=0)
ax.scatter(wvl_list[index]*1e9,fitted_r0[index]*100.,label="Fit",zorder=1,s=25)
ax.tick_params(axis='both', which='major', labelsize=18)
ax.legend()
ax.set_ylim(5,40)
# ax.set_xlim(400,np.max(wvl_list))





# plt.figure(1)
# plt.clf()
# ax = plt.subplot(111)
# plt.plot(wvl_list[index]*1e9,(roptim*(wvl_list[index]/500.*1e9)**(6./5.))*100.,color="C1",linewidth=4,label="Theory",zorder=0)
# plt.scatter(wvl_list[index]*1e9,fitted_r0[index]*100.,label="Fit",zorder=1,s=25)
# #plt.xlabel("Lambda [nm]",fontsize=fs)
# #plt.ylabel("r0 [cm]",fontsize=fs)
# plt.ylim(5,25)
# plt.xlim(400,950)
# plt.grid()
# #plt.fill([laser_min,laser_max,laser_max,laser_min],[plt.ylim()[0],plt.ylim()[0],plt.ylim()[1],plt.ylim()[1]],color="gray",alpha=.5)
# #plt.title("Fried parameter")
# plt.legend()
# plt.xticks()
# plt.yticks()

In [ ]:
conbined_df.columns


In [ ]:
rcParams['figure.figsize'] = 11,9

fig, ax = plt.subplots()
ax.set_ylabel(r'$\mathrm{amplitude [rad^2]}$', fontsize = 18)
ax.set_xlabel(r'$\mathrm{wavelength [nm]}$', fontsize = 18)
ax.scatter(wvl_list[index]*1e9,conbined_df['amplitude'],label="Fit",zorder=1,s=25)
ax.tick_params(axis='both', which='major', labelsize=18)
ax.legend()

In [ ]:
rcParams['figure.figsize'] = 11,9

fig, ax = plt.subplots()
ax.set_ylabel(r'$\mathrm{\alpha}$', fontsize = 18)
ax.set_xlabel(r'$\mathrm{wavelength [nm]}$', fontsize = 18)
ax.scatter(wvl_list[index]*1e9,conbined_df['alpha'],label="Fit",zorder=1,s=25)
ax.tick_params(axis='both', which='major', labelsize=18)
ax.legend()

In [ ]:
rcParams['figure.figsize'] = 11,9

fig, ax = plt.subplots()
ax.set_ylabel(r'$\mathrm{\beta}$', fontsize = 18)
ax.set_xlabel(r'$\mathrm{wavelength [nm]}$', fontsize = 18)
ax.scatter(wvl_list[index]*1e9,conbined_df['beta'],label="Fit",zorder=1,s=25)
ax.tick_params(axis='both', which='major', labelsize=18)
ax.legend()